In [18]:
import traceback
import requests
import pandas as pd
import xml.etree.ElementTree as etree

class GetDataSet:
    url = ''

    def __init__(self, url):
        self.url = url

    def get_oil_price(self):
        try:
            headers = {'user-agent': 'my-app/0.0.1'}
            root = etree.fromstring(requests.get(self.url, headers=headers).text)
            columns = ["型別名稱", "產品編號", "產品名稱", "包裝", "銷售對象", "交貨地點", "計價單位", "參考牌價", "營業稅", "貨物稅", "牌價生效時間", "備註"]
            datatframe = pd.DataFrame(columns = columns)
            content = ''

            for node in root:
                typeName = node.find("型別名稱").text if node is not None else None
                idNum = node.find("產品編號").text if node is not None else None
                prodName = node.find("產品名稱").text if node is not None else None
                package = node.find("包裝").text if node is not None else None
                target = node.find("銷售對象").text if node is not None else None
                local = node.find("交貨地點").text if node is not None else None
                unit = node.find("計價單位").text if node is not None else None
                ref_money = node.find("參考牌價").text if node is not None else None
                tax_1 = node.find("營業稅").text if node is not None else None
                tax_2 = node.find("貨物稅").text if node is not None else None
                time = node.find("牌價生效時間").text if node is not None else None
                note= node.find("備註").text if node is not None else None
                datatframe = datatframe.append(pd.Series([typeName, idNum, prodName, package, target, local,unit,ref_money,tax_1, tax_2, time, note], index = columns), ignore_index = True)
                
                #for Line output string
                content = content + prodName + ':' + unit + ':' + ref_money +' １%0D%0A'
            
            return content
            #return datatframe

        except (ValueError, EOFError, KeyboardInterrupt):
            errorMsg = '數值錯誤!請稍晚再試'
            return errorMsg

        except:
            errorMsg = '資料擷取錯誤，請稍晚再試!'
            return errorMsg
            #traceback.print_exc()
    

In [19]:
test = GetDataSet('https://vipmember.tmtd.cpc.com.tw/opendata/ListPriceWebService.asmx/getCPCMainProdListPrice_XML')
content = test.get_oil_price()
print(content)

98無鉛汽油:元/ 公升:20.8 １%0D%0A95無鉛汽油:元/ 公升:18.8 １%0D%0A92無鉛汽油:元/ 公升:17.3 １%0D%0A酒精汽油:元/ 公升:18.8 １%0D%0A超級柴油:元/ 公升:14.4 １%0D%0A海運輕柴油:元/ 公秉:14400 １%0D%0A海運重柴油:元/ 公秉:13900 １%0D%0A特種低硫燃料油:元/公秉:18620 １%0D%0A甲種低硫燃料油(S:0.5%):元/ 公秉:18118 １%0D%0A低硫燃料油(S:0.5%):元/ 公秉:18118 １%0D%0A發電用低硫燃料油(S:0.5%):元/ 公秉:18118 １%0D%0A低硫高流動點燃料油(S:0.5%):元/ 公秉:18068 １%0D%0A輕裂燃料油:元/ 公秉:18418 １%0D%0A
